In [31]:
#Import Dependencies
import pandas as pd
from datetime import date
from pathlib import Path

In [28]:
#Set Paths for Data CSV's
clinp=Path("Resources/WHO_NREVSS_Clinical_Labs.csv")
pubp=Path("Resources/WHO_NREVSS_Public_Health_Labs.csv")
comp=Path("Resources/WHO_NREVSS_Combined_prior_to_2015_16.csv")

In [99]:
#Read Data from the CSV's
clinical = pd.read_csv(clinp,header=1)
public = pd.read_csv(pubp, header=1)
combo2016= pd.read_csv(comp, header=1)

public.head()

,REGION TYPE,REGION,SEASON_DESCRIPTION,TOTAL SPECIMENS,A (2009 H1N1),A (H3),A (Subtyping not Performed),B,BVic,BYam,H3N2v,A (H5)
0,States,Alabama,Season 2015-16,256,59,16,1,2,2,2,0,0
1,States,Alaska,Season 2015-16,4691,607,98,0,231,1,2,0,0
2,States,Arizona,Season 2015-16,2110,762,580,0,13,58,399,0,0
3,States,Arkansas,Season 2015-16,128,20,8,0,1,12,0,0,0
4,States,California,Season 2015-16,12820,1462,854,35,775,309,803,0,0


In [ ]:
#Isolate the data taken during flu season (October-May)
clinical["Season"]=clinical[["YEAR","WEEK"]].apply(lambda row: date.fromisocalendar(int(row['YEAR']), int(row['WEEK']),1).month,axis=1)
clinical=clinical.drop(clinical[(clinical.Season<10)&(clinical.Season>5)].index)

#Convert date data into "Season year1-year2" format present in the Public dataset
clinical.loc[(clinical["Season"]<10),"YEAR"]=clinical["YEAR"].apply(lambda x: x-1)
clinical["YEAR_2"]=clinical["YEAR"].apply(lambda x: str(x+1)[2:4])
clinical["Season"]=clinical[["YEAR","YEAR_2"]].apply(lambda row: "Season "+str(row["YEAR"])+"-"+row["YEAR_2"],axis=1)
clinical["Season"].head()


54    Season 2015-16
55    Season 2015-16
56    Season 2015-16
57    Season 2015-16
58    Season 2015-16
Name: Season, dtype: object